# SmoothLife

## A first description of gliders in continuous CA 

<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/smoothlife_glider_summary.png"> -->


### Overview

<!-- local image -->
![Adorbium cellular automaton gliders](../assets/smoothlife_glider_summary.png)


$$
A_{t+ \Delta t} = A_t + \Delta t * U(A_t)
$$

### Neighborhoods

$$
K_i = \left\{ 
        \begin{array}{ c l }
            \frac{1}{M} & \quad \textrm{if } r < r_i \\
            0                 & \quad \textrm{otherwise}
            \end{array}
            \right.
$$

$$
K_n = \left\{ 
        \begin{array}{ c l }
            \frac{1}{N} & \quad \textrm{if } r_i \leq r < r_o\\
            0                 & \quad \textrm{otherwise}
            \end{array}
            \right.

$$


### Updates

$$
s_{\mu, \alpha}(x) = \frac{1}{1+e^{-\frac{x-\mu}{\alpha}}}
$$

$$
G(x) = s_{\mu_{gl}, \alpha_g}(x) * (1- s_{\mu_{gu}})
$$

$$
P(x) = s_{\mu_{pl}, \alpha_p}(x) * (1- s_{\mu_{pu}})
$$


Where $\mu_{gl}, \mu_{gu}$ and $\mu_{pl}, \mu_{pu}$ define the lower and upper edges of the genesis and persistence functions, respectively. The $\alpha$ variable modulates the smoothness of the edges. We use only one set of parameters for SmoothLife in this work: $\alpha_g=0.0280$, $\alpha_p=0.1470$, $\mu_{gl}=0.2780$, $\mu_{gu}=0.3650$, $\mu_{pl}=0.2670$, and $\mu_{pu}=0.4450$


<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/smoothlife_4.gif"> -->

<!-- local image -->
![Adorbium cellular automaton gliders](../assets/smoothlife_4.gif)

In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="Adamata animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(grid[0,0])[:,:,:3]
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    #global ax
    
    grid = aa(grid)
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(grid[0,0])[:,:,:3]
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()